In [1]:
# """
# You can add your own functions here according to your decision tree implementation.
# There is no restriction on following the below template, these fucntions are here to simply help you.
# """

In [9]:
import pandas as pd
import numpy as np
from scipy.special import xlogy
from sklearn.preprocessing import OneHotEncoder

In [2]:
def one_hot_encoding(X: pd.DataFrame) -> pd.DataFrame:
    """
    Function to perform one hot encoding on the input data
    """
    # Initialize OneHotEncoder
    encoder = OneHotEncoder(sparse=False)

    # Fit and transform the data
    encoded_array = encoder.fit_transform(X)
    encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(X.columns))
    return encoded_df


In [3]:
def check_ifreal(y: pd.Series) -> bool:
    """
    Function to check if the given series has real or discrete values
    """
    
    """
    
    """
    
    if(pd.api.types.is_bool_dtype(y)):
        return False
    elif (pd.api.types.is_any_real_numeric_dtype(y)):
        unique_ration = y.nunique()/y.len()
        if(unique_ration<0.05):
            return False
        else:
            return True
    elif (pd.api.types.is_object_dtype(y)):
        unique_ration = y.nunique()/y.len()
        if(unique_ration<0.05):
            return False
        else:
            return True
    else :
        
        return False
    
    



In [4]:
def entropy(Y: pd.Series) -> float:
    """
    Function to calculate the entropy
    """
    if (check_ifreal(Y)==False):
        prob = Y.unique()
        val_cnts = Y.value_counts()
        # tot_cnt = Y.value_counts().sum()
        entropy = 0.0
        for y in val_cnts:
            proportion = y/len(Y)
            # entropy+= proportion*np.log2(proportion)
            entropy+=xlogy(proportion,proportion)/np.log(2)
        entropy=-entropy
        return entropy   
    else:
        mean = Y.mean()
        mse = ((Y- mean) ** 2).mean()
        return mse
        

    



In [5]:
def gini_index(Y: pd.Series) -> float:
    """
    Function to calculate the gini index
    """
    if(check_ifreal(Y)==False):  # non numeric values
        pob = Y.unique()
        val_cnts = Y.value_counts()
        gi=0
     
        for y in val_cnts:
            proportion = (y/len(Y))**2
            gi = gi + proportion
        return 1-gi


    

In [6]:
def information_gain(Y: pd.Series, attr: pd.Series, criterion: str) -> float:
    """
    Function to calculate the information gain using criterion (entropy, gini index or MSE)
    """

    pass


In [7]:
def opt_split_attribute(X: pd.DataFrame, y: pd.Series, criterion, features: pd.Series):
    """
    Function to find the optimal attribute to split about.
    If needed you can split this function into 2, one for discrete and one for real valued features.
    You can also change the parameters of this function according to your implementation.

    features: pd.Series is a list of all the attributes we have to split upon

    return: attribute to split upon
    """

    # According to wheather the features are real or discrete valued and the criterion, find the attribute from the features series with the maximum information gain (entropy or varinace based on the type of output) or minimum gini index (discrete output).

    pass


In [8]:
def split_data(X: pd.DataFrame, y: pd.Series, attribute, value):
    """
    Funtion to split the data according to an attribute.
    If needed you can split this function into 2, one for discrete and one for real valued features.
    You can also change the parameters of this function according to your implementation.

    attribute: attribute/feature to split upon
    value: value of that attribute to split upon

    return: splitted data(Input and output)
    """

    # Split the data based on a particular value of a particular attribute. You may use masking as a tool to split the data.

    pass